In [ ]:
import warnings, os
import datetime as dt
from dateutil.relativedelta import relativedelta as rtd
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin([*SYMBOLS]) & Raw['tf'].isin([*TFRAMES])] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    Src['Week.Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['Hilo']     = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Day Open']        = Calc_by['Open'].transform('first')
    Calc['ID High']         = Calc_by['High'].expanding().max()     .reset_index(drop=1)
    Calc['ID Low']          = Calc_by['Low'].expanding().min()      .reset_index(drop=1)

    Calc['ID Change']       = Calc['Close']   - Calc['Day Open']
    Calc['ID Hilo']         = Calc['ID High'] - Calc['ID Low']

    Calc['ID Sign']         = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Calc['ID Sign Bull']    = Calc['ID Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['ID Sign Bear']    = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['ID Chg Abs']      = Calc['ID Change'].abs()
    Calc['ID Chg Pos']      = Calc['ID Chg Abs'] * Calc['ID Sign Bull'] 
    Calc['ID Chg Neg']      = Calc['ID Chg Abs'] * Calc['ID Sign Bear'] 

    Calc['ID HL Pos']       = Calc['ID Hilo'] * Calc['ID Sign Bull']
    Calc['ID HL Neg']       = Calc['ID Hilo'] * Calc['ID Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [269]:
def STATS(Calc, Col):
    pipe = []
    for HR in [9,10,11,12,13,14,15,16,17,18]: 

        Df = Calc[Calc['Time'] <= dt.time(HR,00)]

        pipe.append({
            'Before': dt.time(HR+1,00), 
            **Df[Col].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)

In [270]:
STATS(Calc, Col='ID Change')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1241,-6,608,-3113,-377,16,351,2811
1,11:00:00,2482,5,812,-3223,-462,6,474,6093
2,12:00:00,3723,-6,949,-5087,-550,0,542,6093
3,13:00:00,4964,-7,1046,-5461,-621,-6,604,6093
4,14:00:00,6210,-8,1112,-5461,-680,0,663,6093
5,15:00:00,7455,-8,1170,-5461,-713,0,706,6093
6,16:00:00,8700,-10,1225,-6707,-743,0,738,6093
7,17:00:00,9945,-11,1276,-6707,-770,6,769,6295
8,18:00:00,11190,-13,1325,-6707,-800,8,791,6295
9,19:00:00,11866,-12,1342,-7641,-812,14,810,6295


In [271]:
STATS(Calc, Col='ID Chg Abs')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1241,460,398,0,163,361,647,3113
1,11:00:00,2482,609,536,0,216,464,872,6093
2,12:00:00,3723,711,628,0,250,545,1005,6093
3,13:00:00,4964,789,687,0,275,614,1118,6093
4,14:00:00,6210,844,723,0,292,674,1201,6093
5,15:00:00,7455,892,758,0,312,710,1270,6093
6,16:00:00,8700,933,794,0,326,740,1333,6707
7,17:00:00,9945,972,826,0,342,770,1390,6707
8,18:00:00,11190,1008,860,0,354,795,1435,6707
9,19:00:00,11866,1021,872,0,359,810,1448,7641


In [272]:
STATS(Calc, Col='ID Chg Pos')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,638,442,387,0,138,344,649,2811
1,11:00:00,1254,608,541,0,203,461,890,6093
2,12:00:00,1864,704,620,0,236,540,1026,6093
3,13:00:00,2477,783,677,0,264,606,1147,6093
4,14:00:00,3109,835,712,0,279,660,1226,6093
5,15:00:00,3743,880,742,0,297,700,1293,6093
6,16:00:00,4377,918,770,0,309,732,1351,6093
7,17:00:00,5018,953,798,0,324,758,1395,6295
8,18:00:00,5660,983,827,0,338,780,1435,6295
9,19:00:00,6018,994,836,0,343,792,1447,6295


In [273]:
STATS(Calc, Col='ID Chg Neg')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,603,-479,408,-3113,-644,-383,-187,-5
1,11:00:00,1228,-611,532,-3223,-858,-469,-229,-5
2,12:00:00,1859,-718,637,-5087,-993,-550,-262,-5
3,13:00:00,2487,-795,697,-5461,-1092,-620,-284,-5
4,14:00:00,3101,-853,735,-5461,-1183,-680,-306,-5
5,15:00:00,3712,-903,774,-5461,-1243,-717,-327,-5
6,16:00:00,4323,-949,817,-6707,-1312,-749,-342,-5
7,17:00:00,4927,-992,853,-6707,-1379,-777,-354,-5
8,18:00:00,5530,-1033,891,-6707,-1433,-814,-370,-5
9,19:00:00,5848,-1048,906,-7641,-1455,-826,-378,-5


In [274]:
STATS(Calc, Col='ID Hilo')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1241,943,437,241,630,858,1143,3852
1,11:00:00,2482,1220,609,241,780,1103,1525,7321
2,12:00:00,3723,1417,720,241,899,1283,1772,7731
3,13:00:00,4964,1563,793,241,1000,1420,1965,7731
4,14:00:00,6210,1671,839,241,1069,1524,2092,7731
5,15:00:00,7455,1764,884,241,1126,1614,2211,7731
6,16:00:00,8700,1848,927,241,1179,1684,2312,7731
7,17:00:00,9945,1924,971,241,1229,1745,2407,7849
8,18:00:00,11190,1991,1009,241,1268,1807,2488,7864
9,19:00:00,11866,2010,1022,241,1285,1821,2510,8487


In [275]:
STATS(Calc, Col='ID HL Pos')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,638,936,422,241,626,861,1143,3236
1,11:00:00,1254,1225,607,241,788,1112,1548,7321
2,12:00:00,1864,1415,707,241,902,1290,1798,7731
3,13:00:00,2477,1555,775,241,997,1438,1971,7731
4,14:00:00,3109,1659,816,241,1075,1541,2087,7731
5,15:00:00,3743,1748,859,241,1126,1618,2198,7731
6,16:00:00,4377,1828,899,241,1174,1685,2294,7731
7,17:00:00,5018,1899,938,241,1225,1743,2392,7849
8,18:00:00,5660,1961,970,241,1259,1811,2468,7864
9,19:00:00,6018,1980,983,241,1268,1820,2491,7864


In [276]:
STATS(Calc, Col='ID HL Neg')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,603,-950,452,-3852,-1144,-851,-635,-309
1,11:00:00,1228,-1215,610,-4852,-1499,-1086,-773,-309
2,12:00:00,1859,-1419,733,-6308,-1744,-1276,-892,-309
3,13:00:00,2487,-1570,810,-6775,-1956,-1408,-1004,-309
4,14:00:00,3101,-1684,862,-6775,-2102,-1511,-1065,-309
5,15:00:00,3712,-1781,908,-6775,-2235,-1609,-1128,-309
6,16:00:00,4323,-1868,955,-7479,-2334,-1684,-1185,-309
7,17:00:00,4927,-1949,1002,-7753,-2424,-1748,-1232,-309
8,18:00:00,5530,-2021,1046,-7753,-2511,-1803,-1282,-309
9,19:00:00,5848,-2041,1060,-8487,-2534,-1824,-1299,-309


# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Day Open']        = Calc_by['Open'].transform('first')
    Calc['ID High']         = Calc_by['High'].expanding().max()     .reset_index(drop=1)
    Calc['ID Low']          = Calc_by['Low'].expanding().min()      .reset_index(drop=1)

    Calc['ID Change']       = Calc['Close']   - Calc['Day Open']
    Calc['ID Hilo']         = Calc['ID High'] - Calc['ID Low']

    Calc['ID Sign']         = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Calc['ID Sign Bull']    = Calc['ID Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['ID Sign Bear']    = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['ID Chg Abs']      = Calc['ID Change'].abs()
    Calc['ID Chg Pos']      = Calc['ID Chg Abs'] * Calc['ID Sign Bull'] 
    Calc['ID Chg Neg']      = Calc['ID Chg Abs'] * Calc['ID Sign Bear'] 

    Calc['ID HL Pos']       = Calc['ID Hilo'] * Calc['ID Sign Bull']
    Calc['ID HL Neg']       = Calc['ID Hilo'] * Calc['ID Sign Bear']
    return Calc


def STATS(Calc, Col):
    pipe = []
    for HR in [9,10,11,12,13,14,15,16,17,18]: 
        Df = Calc[Calc['Time'] <= dt.time(HR,00)]
        pipe.append({
            'Before': dt.time(HR+1,00), 
            **Df[Col].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)